In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import joblib

data = pd.read_csv('/content/encoded_fertilizer_dataset.csv')
data.fillna(data.mean(), inplace=True)

features = [
    'Moisture_Content', 'pH_Level', 'Carbon_Content', 'Nitrogen_Content',
    'Phosphorus_Content', 'Potassium_Content', 'Microbial_Activity',
    'Processing_Temperature', 'Processing_Time', 'Energy_Consumption'
]
fertility_target = 'Fertilizer_Quality'

X = data[features]
y_fertility = data[fertility_target]

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_fertility, test_size=0.2, random_state=42)

knn = KNeighborsRegressor(n_neighbors=5)
linear_reg = LinearRegression()
rf = RandomForestRegressor(random_state=42, n_estimators=100)

def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

X_train_lstm = X_train.values.reshape(-1, X_train.shape[1], 1)
X_test_lstm = X_test.values.reshape(-1, X_train.shape[1], 1)
lstm_model = create_lstm_model()
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=16, verbose=1)
lstm_predictions = lstm_model.predict(X_test_lstm).flatten()

stacking_regressor = StackingRegressor(
    estimators=[
        ('knn', knn),
        ('linear', linear_reg),
        ('random_forest', rf)
    ],
    final_estimator=GradientBoostingRegressor()
)

stacking_regressor.fit(X_train, y_train)
stacking_predictions = stacking_regressor.predict(X_test)

lstm_weight = 0.4
stacking_weight = 0.6
final_predictions = (lstm_weight * lstm_predictions + stacking_weight * stacking_predictions)

rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
r2 = r2_score(y_test, final_predictions)
print(f"Stacked Model RMSE: {rmse}")
print(f"Stacked Model R²: {r2}")

joblib.dump(stacking_regressor, 'stacking_regressor.pkl')
lstm_model.save('lstm_model.h5')

def generate_report():
    user_input = {}
    for feature in features:
        user_input[feature] = float(input(f"Enter value for {feature}: "))

    input_normalized = scaler.transform(pd.DataFrame([user_input], columns=features))

    predicted_fertility = stacking_regressor.predict(input_normalized)[0]
    predicted_lstm = lstm_model.predict(input_normalized.reshape(-1, X_train.shape[1], 1)).flatten()[0]

    final_prediction = (lstm_weight * predicted_lstm + stacking_weight * predicted_fertility)

    fertility_recommendations = []
    if user_input['Nitrogen_Content'] < 2:
        fertility_recommendations.append("Increase Nitrogen levels.")
    if user_input['Phosphorus_Content'] < 1:
        fertility_recommendations.append("Increase Phosphorus levels.")
    if user_input['Potassium_Content'] < 1:
        fertility_recommendations.append("Increase Potassium levels.")
    if user_input['pH_Level'] < 6.0 or user_input['pH_Level'] > 7.5:
        fertility_recommendations.append("Adjust soil pH to the range 6.0-7.5.")

    print("Soil Fertility Report:")
    print(f"Predicted Fertilizer Quality: {final_prediction:.2f}")
    print("Fertility Recommendations:")
    for recommendation in fertility_recommendations:
        print(f"- {recommendation}")

generate_report()
